# Advanced Transpilation - Code Laboratory

**Section 4: Running Circuits on IBM Quantum Hardware** | [See README for concepts](./README.md)

---

## 🔧 Quick API Reference

| Method | Signature | Returns | Use When |
|--------|-----------|---------|----------|
| `generate_preset_pass_manager()` | `generate_preset_pass_manager(level, backend, layout_method, routing_method)` | `PassManager` | Full control over transpilation |
| `PassManager()` | `PassManager(passes)` | `PassManager` | Custom optimization pipeline |
| `TransformationPass` | Custom class extending `TransformationPass` | Modified DAG | Creating custom passes |
| `pm.run()` | `pm.run(circuit)` | `QuantumCircuit` | Apply pass manager |

---

In [ ]:
"""
Qiskit Code Laboratory - Advanced Transpilation
================================================
Prerequisites: See README.md for conceptual background
"""

# Standard imports
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.transpiler import PassManager, TransformationPass
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler.passes import Optimize1qGates, InverseCancellation
from qiskit.dagcircuit import DAGCircuit
from qiskit.circuit.library import HGate, XGate, CXGate

# IBM Quantum Runtime imports
from qiskit_ibm_runtime.fake_provider import FakeManilaV2, FakeKyoto

# =============================================================
# UTILITY FUNCTIONS FOR THIS NOTEBOOK
# =============================================================

def compare_transpilation(qc, backend, **kwargs):
    """Compare transpilation with different options."""
    pm = generate_preset_pass_manager(backend=backend, **kwargs)
    t = pm.run(qc)
    return {
        'depth': t.depth(),
        'size': t.size(),
        'cx_count': t.count_ops().get('cx', 0)
    }

def show_layout(transpiled_circuit):
    """Display layout mapping from transpiled circuit."""
    layout = transpiled_circuit.layout
    print(f"Initial layout: {layout.initial_layout}")
    print(f"Final layout: {layout.final_layout}")

print("✅ Environment ready - using FakeManilaV2 for demonstrations")

---

## `generate_preset_pass_manager()` - Advanced Options

### Signature
```python
generate_preset_pass_manager(
    optimization_level: int,      # 0-3
    backend: Backend = None,
    layout_method: str = None,    # 'trivial', 'dense', 'sabre'
    routing_method: str = None,   # 'basic', 'stochastic', 'sabre'
    scheduling_method: str = None, # 'asap', 'alap'
    approximation_degree: float = 1.0,
    seed_transpiler: int = None
) -> PassManager
```

### Key Parameters
| Parameter | Options | Description |
|-----------|---------|-------------|
| `layout_method` | `'trivial'`, `'dense'`, `'sabre'` | How to map logical→physical qubits |
| `routing_method` | `'basic'`, `'stochastic'`, `'sabre'` | How to handle non-adjacent qubits |
| `scheduling_method` | `'asap'`, `'alap'` | Timing alignment strategy |

### Returns
`PassManager` with configurable 6-stage pipeline

### See Also
- [README: Transpilation Pipeline](./README.md#transpilation)

In [ ]:
# ============================================================
# generate_preset_pass_manager() - BASIC USAGE (Full Options)
# ============================================================

backend = FakeManilaV2()

# Create pass manager with explicit method choices
pm = generate_preset_pass_manager(
    optimization_level=2,
    backend=backend,
    layout_method='sabre',
    routing_method='sabre',
    seed_transpiler=42
)

qc = QuantumCircuit(3)
qc.h(0)
qc.cx(0, 1)
qc.cx(1, 2)
qc.cx(0, 2)  # May require routing

transpiled = pm.run(qc)

print("Transpiled with full options:")
print(f"Depth: {transpiled.depth()}, Size: {transpiled.size()}")
print(f"CX gates: {transpiled.count_ops().get('cx', 0)}")

print("\n✅ generate_preset_pass_manager() gives fine-grained control")

In [ ]:
# ============================================================
# LAYOUT METHODS COMPARISON
# ============================================================

backend = FakeManilaV2()

qc = QuantumCircuit(3)
qc.h(0)
qc.cx(0, 1)
qc.cx(1, 2)
qc.cx(0, 2)

print("Layout Method Comparison:")
print("=" * 55)

for method in ['trivial', 'dense', 'sabre']:
    pm = generate_preset_pass_manager(
        optimization_level=2,
        backend=backend,
        layout_method=method,
        seed_transpiler=42
    )
    t = pm.run(qc)
    cx = t.count_ops().get('cx', 0)
    print(f"{method:10}: depth={t.depth():2d}, CX={cx}")

print("\n💡 'sabre' is usually best for general circuits")

In [ ]:
# ============================================================
# ROUTING METHODS COMPARISON
# ============================================================

backend = FakeManilaV2()

# Circuit requiring routing (far-apart qubits)
qc = QuantumCircuit(5)
qc.h(0)
qc.cx(0, 4)  # Far apart on linear topology!
qc.cx(1, 3)

print("Routing Method Comparison:")
print("=" * 55)

for method in ['basic', 'stochastic', 'sabre']:
    pm = generate_preset_pass_manager(
        optimization_level=2,
        backend=backend,
        routing_method=method,
        seed_transpiler=42
    )
    t = pm.run(qc)
    cx = t.count_ops().get('cx', 0)
    print(f"{method:12}: depth={t.depth():2d}, CX={cx}")

print("\n💡 'sabre' balances quality and speed")

---

## `PassManager` - Custom Pipeline

### Signature
```python
PassManager(passes: List[BasePass] = None)
```

### Key Passes
| Pass | Purpose | Example |
|------|---------|---------|
| `Optimize1qGates` | Merge consecutive 1-qubit gates | `Rz·Rz → Rz` |
| `InverseCancellation` | Cancel gate-inverse pairs | `H·H = I` |
| `CommutativeCancellation` | Cancel commuting gates | Z optimization |

### Returns
`PassManager` - Reusable custom optimization pipeline

### See Also
- [README: Custom Passes](./README.md#transpilation)

In [ ]:
# ============================================================
# PassManager - BASIC USAGE
# ============================================================

# Create custom pipeline with standard passes
custom_pm = PassManager([
    Optimize1qGates(),
    InverseCancellation([HGate(), XGate(), CXGate()])
])

# Circuit with redundant gates
qc = QuantumCircuit(2)
qc.h(0)
qc.h(0)  # H·H = I, should cancel
qc.x(1)
qc.x(1)  # X·X = I, should cancel
qc.cx(0, 1)

print("Original circuit:")
print(qc.draw())
print(f"Gates: {sum(qc.count_ops().values())}")

optimized = custom_pm.run(qc)

print("\nAfter custom PassManager:")
print(optimized.draw())
print(f"Gates: {sum(optimized.count_ops().values())}")

print("\n✅ PassManager applies passes in order")

---

## `TransformationPass` - Custom Pass Creation

### Signature
```python
class MyPass(TransformationPass):
    def run(self, dag: DAGCircuit) -> DAGCircuit:
        # Modify DAG here
        return dag
```

### Pass Types
| Type | Base Class | Modifies Circuit? |
|------|------------|-------------------|
| Transformation | `TransformationPass` | Yes |
| Analysis | `AnalysisPass` | No |

### See Also
- [README: Custom Passes](./README.md#transpilation)

In [ ]:
# ============================================================
# TransformationPass - CUSTOM PASS CREATION
# ============================================================

class RemoveBarriers(TransformationPass):
    """Custom pass to remove all barrier instructions."""
    
    def run(self, dag: DAGCircuit) -> DAGCircuit:
        nodes_to_remove = [
            node for node in dag.op_nodes() 
            if node.name == 'barrier'
        ]
        for node in nodes_to_remove:
            dag.remove_op_node(node)
        return dag

# Test with barriers
qc = QuantumCircuit(2)
qc.h(0)
qc.barrier()
qc.cx(0, 1)
qc.barrier()
qc.h(1)

print("Original (with barriers):")
print(qc.draw())

pm = PassManager([RemoveBarriers()])
cleaned = pm.run(qc)

print("\nAfter RemoveBarriers:")
print(cleaned.draw())

print("\n✅ TransformationPass modifies DAG, returns modified DAG")

In [ ]:
# ============================================================
# ⚠️ TRAP DEMONSTRATION: Pass Order Matters
# ============================================================

print("⚠️ TRAP: Pass order affects optimization results!")
print("=" * 55)

# Circuit with multiple redundancies
qc = QuantumCircuit(2)
qc.h(0)
qc.rz(0.1, 0)
qc.h(0)  # H·Rz·H can be simplified
qc.x(1)
qc.x(1)

# Order 1: InverseCancellation first
pm1 = PassManager([
    InverseCancellation([HGate(), XGate()]),
    Optimize1qGates()
])

# Order 2: Optimize1qGates first  
pm2 = PassManager([
    Optimize1qGates(),
    InverseCancellation([HGate(), XGate()])
])

r1 = pm1.run(qc)
r2 = pm2.run(qc)

print(f"Original gates: {sum(qc.count_ops().values())}")
print(f"InverseCancellation→Optimize1qGates: {sum(r1.count_ops().values())} gates")
print(f"Optimize1qGates→InverseCancellation: {sum(r2.count_ops().values())} gates")

print("\n💡 Best order: Optimize1qGates → InverseCancellation → Optimize1qGates")

In [ ]:
# ============================================================
# ⚠️ TRAP DEMONSTRATION: Layout Affects Observables
# ============================================================

from qiskit.quantum_info import SparsePauliOp

print("⚠️ TRAP: Transpilation changes qubit mapping!")
print("=" * 55)

backend = FakeManilaV2()
pm = generate_preset_pass_manager(optimization_level=2, backend=backend)

qc = QuantumCircuit(3)
qc.h(0)
qc.cx(0, 1)
qc.cx(1, 2)

transpiled = pm.run(qc)

print("Layout mapping:")
show_layout(transpiled)

# Observable on LOGICAL qubits
observable = SparsePauliOp("ZZI")  # Logical qubits 0,1

# MUST apply layout before Estimator!
mapped_obs = observable.apply_layout(transpiled.layout)

print(f"\nOriginal observable: {observable}")
print(f"Mapped observable: {mapped_obs}")

print("\n💡 ALWAYS use observable.apply_layout() before Estimator!")

In [ ]:
# ============================================================
# CHALLENGE 1: Custom Optimization Pipeline
# ============================================================
# Task: Create a PassManager that removes barriers AND cancels inverses
# Expected: Combined optimization in single pipeline
# ============================================================

def create_cleanup_pipeline():
    """
    Create a PassManager that:
    1. Removes all barriers
    2. Cancels H·H and X·X
    
    Returns:
        PassManager ready to use
    """
    return PassManager([
        RemoveBarriers(),  # Our custom pass from above
        InverseCancellation([HGate(), XGate()])
    ])

# Test circuit
qc_test = QuantumCircuit(2)
qc_test.h(0)
qc_test.barrier()
qc_test.h(0)  # Should cancel with first H
qc_test.x(1)
qc_test.barrier()
qc_test.x(1)  # Should cancel

cleanup_pm = create_cleanup_pipeline()
cleaned = cleanup_pm.run(qc_test)

print("Challenge 1: Custom Cleanup Pipeline")
print("=" * 50)
print(f"Original gates: {sum(qc_test.count_ops().values())}")
print(f"Cleaned gates: {sum(cleaned.count_ops().values())}")

assert cleaned.count_ops().get('barrier', 0) == 0, "Barriers should be removed"
assert 'h' not in cleaned.count_ops(), "H gates should cancel"
print("\n✅ Challenge 1 PASSED - Custom pipeline works!")

In [ ]:
# ============================================================
# CHALLENGE 2: Compare Layout Methods on Complex Circuit
# ============================================================
# Task: Find the best layout method for a 4-qubit circuit
# Expected: Demonstrate method impact on circuit metrics
# ============================================================

def find_best_layout(qc, backend, methods=['trivial', 'dense', 'sabre']):
    """
    Compare layout methods and return best one.
    
    Returns:
        Dict with method name and metrics
    """
    results = {}
    for method in methods:
        pm = generate_preset_pass_manager(
            optimization_level=2,
            backend=backend,
            layout_method=method,
            seed_transpiler=42
        )
        t = pm.run(qc)
        results[method] = {
            'depth': t.depth(),
            'cx': t.count_ops().get('cx', 0)
        }
    
    # Find method with lowest CX count
    best = min(results, key=lambda m: results[m]['cx'])
    return best, results

# Complex circuit needing layout optimization
backend = FakeManilaV2()
qc = QuantumCircuit(4)
qc.h([0, 1, 2, 3])
qc.cx(0, 2)  # Non-adjacent
qc.cx(1, 3)  # Non-adjacent
qc.cx(0, 3)  # Far apart

best_method, all_results = find_best_layout(qc, backend)

print("Challenge 2: Layout Method Comparison")
print("=" * 50)
for method, metrics in all_results.items():
    marker = "← BEST" if method == best_method else ""
    print(f"{method:10}: depth={metrics['depth']:2d}, CX={metrics['cx']:2d} {marker}")

assert best_method in ['sabre', 'dense'], "Optimized method should win"
print(f"\n✅ Challenge 2 PASSED - Best method: {best_method}")

---

## Observable Layout Mapping (apply_layout)

### Key Concept
After transpilation, virtual qubits are remapped to physical qubits. Observables MUST be remapped to match!

### Pattern
```python
transpiled = pm.run(circuit)
mapped_obs = observable.apply_layout(transpiled.layout)
pub = (transpiled, mapped_obs)
```

### Trap
❌ Using original observable after transpilation → Wrong qubits!
✅ Using `observable.apply_layout(transpiled.layout)` → Correct mapping

### See Also
- [README: Layout Remapping](./README.md#advanced-transpilation)

In [ ]:
# ============================================================
# ⚠️ TRAP DEMONSTRATION: apply_layout() for Observables
# ============================================================

from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime.fake_provider import FakeManilaV2

print("⚠️ TRAP: Observable Layout Remapping")
print("=" * 55)

# Create circuit
backend = FakeManilaV2()
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

# Transpile
pm = generate_preset_pass_manager(optimization_level=2, backend=backend, seed_transpiler=42)
transpiled = pm.run(qc)

# Original observable
observable = SparsePauliOp('ZZ')

print("Before transpilation:")
print(f"  Circuit uses virtual qubits 0, 1")
print(f"  Observable: {observable}")

print("\nAfter transpilation:")
print(f"  Initial layout: {transpiled.layout.initial_layout}")
print(f"  Qubits may be remapped!")

print("""
❌ WRONG:
   pub = (transpiled, observable)  # Observable on wrong qubits!

✅ CORRECT:
   mapped_obs = observable.apply_layout(transpiled.layout)
   pub = (transpiled, mapped_obs)  # Observable on correct physical qubits!
""")

# Demonstrate correct approach
mapped_obs = observable.apply_layout(transpiled.layout)
print(f"Mapped observable: {mapped_obs}")
print(f"\n💡 ALWAYS apply_layout() after manual transpilation!")

---

## Broadcasting Rules (Parameter/Observable Arrays)

### Key Concept
Primitives use NumPy-style broadcasting to combine parameter and observable arrays efficiently.

### Broadcasting Patterns
| Pattern | Params Shape | Obs Shape | Result Shape |
|---------|--------------|-----------|--------------|
| Single obs, many params | `(5,)` | `()` | `(5,)` |
| Zip (1-to-1) | `(5,)` | `(5,)` | `(5,)` |
| Outer product | `(1, 5)` | `(3, 1)` | `(3, 5)` |

### Rule: Shapes must be broadcastable (equal or one is 1)

### See Also
- [README: Broadcasting](./README.md#broadcasting-rules)

In [ ]:
# ============================================================
# BROADCASTING RULES - Parameter/Observable Arrays
# ============================================================

import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import StatevectorEstimator

print("Broadcasting Rules for Primitives")
print("=" * 60)

# Create parameterized circuit
theta = Parameter('θ')
qc = QuantumCircuit(2)
qc.ry(theta, 0)
qc.cx(0, 1)

estimator = StatevectorEstimator()

# Pattern 1: Single observable, multiple parameters (broadcasts)
print("\n1️⃣ Pattern 1: Single obs → multiple params (broadcast)")
params = np.linspace(0, np.pi, 5)  # shape (5,)
observable = SparsePauliOp('ZZ')   # scalar
pub = (qc, observable, params)
result = estimator.run([pub]).result()
print(f"  Params shape: (5,)")
print(f"  Observable: scalar")
print(f"  Result shape: {result[0].data.evs.shape}")

# Pattern 2: Matching shapes (zip)
print("\n2️⃣ Pattern 2: Same shape → zip (1-to-1)")
params = np.array([0.0, np.pi/2])  # shape (2,)
observables = [SparsePauliOp('ZZ'), SparsePauliOp('XX')]  # shape (2,)
pub = (qc, observables, params)
result = estimator.run([pub]).result()
print(f"  Params shape: (2,)")
print(f"  Observables shape: (2,)")
print(f"  Result shape: {result[0].data.evs.shape}")
print(f"  Result: ⟨ZZ⟩(0)={result[0].data.evs[0]:.3f}, ⟨XX⟩(π/2)={result[0].data.evs[1]:.3f}")

# Pattern 3: Outer product
print("\n3️⃣ Pattern 3: (M,1) × (1,N) → outer product (M×N)")
params = np.linspace(0, np.pi, 3).reshape(1, 3)  # shape (1, 3)
observables = [[SparsePauliOp('ZZ')], [SparsePauliOp('XX')]]  # shape (2, 1)
pub = (qc, observables, params)
result = estimator.run([pub]).result()
print(f"  Params shape: (1, 3)")
print(f"  Observables shape: (2, 1)")
print(f"  Result shape: {result[0].data.evs.shape}")

print("\n💡 RULE: Shapes must be broadcastable (equal or one is 1)")

In [ ]:
# ============================================================
# ⚠️ TRAP DEMONSTRATION: Incompatible Broadcasting Shapes
# ============================================================

import numpy as np
from qiskit.quantum_info import SparsePauliOp

print("⚠️ TRAP: Incompatible Broadcasting Shapes")
print("=" * 55)

print("""
❌ INCOMPATIBLE SHAPES:
   params shape: (5,)
   observables shape: (3,)
   → Error! 5 and 3 are incompatible

✅ COMPATIBLE SHAPES (examples):
   (5,) + ()      → (5,)    # Scalar broadcasts
   (5,) + (5,)    → (5,)    # Same size = zip
   (1,5) + (3,1)  → (3,5)   # Outer product
   (5,) + (1,)    → (5,)    # 1 broadcasts to 5
""")

# Demo shape checking
def check_broadcast_shape(params_shape, obs_shape):
    """Check if shapes are broadcastable."""
    try:
        result = np.broadcast_shapes(params_shape, obs_shape)
        print(f"  {params_shape} + {obs_shape} → {result} ✓")
        return True
    except ValueError:
        print(f"  {params_shape} + {obs_shape} → ERROR! ✗")
        return False

print("\nShape compatibility test:")
check_broadcast_shape((5,), ())      # OK - scalar
check_broadcast_shape((5,), (5,))    # OK - same
check_broadcast_shape((1,5), (3,1))  # OK - outer product
check_broadcast_shape((5,), (3,))    # ERROR!

print("\n💡 TIP: Use reshape() to make shapes compatible")